In [1]:
!pip install torch torchvision

In [2]:
!pip install facenet-pytorch

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image

In [4]:
!pip install -q kaggle

In [5]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"andreeahuzum","key":"24f14f8deadb91cefd0ee885481a5b2a"}'}

In [6]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d msambare/fer2013

Dataset URL: https://www.kaggle.com/datasets/msambare/fer2013
License(s): DbCL-1.0
fer2013.zip: Skipping, found more recently modified local copy (use --force to force download)


In [8]:
!unzip fer2013.zipN

Archive:  fer2013.zip
replace test/angry/PrivateTest_10131363.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [9]:
train_dir = '/content/train'
test_dir = '/content/test'

In [10]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  #grayscale
    transforms.Resize((48, 48)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.ImageFolder('/content/train', transform=transform)

test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_dataset = datasets.ImageFolder('/content/test', transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [11]:
class ER_CNN(nn.Module):
    def __init__(self):
        super(ER_CNN, self).__init__()

        self.conv_block1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.1),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.1)
        )

        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1)
        )

        self.conv_block3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1)
        )

        self.conv_block4 = nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1)
        )

        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(256 * 3 * 3, 512)
        self.dropout1 = nn.Dropout(0.4)
        self.fc2 = nn.Linear(512, 256)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(256, 7)  # 7 clase

    def forward(self, x):
        x = self.pool(self.conv_block1(x))  # 48 -> 24
        x = self.pool(self.conv_block2(x))  # 24 -> 12
        x = self.pool(self.conv_block3(x))  # 12 -> 6
        x = self.pool(self.conv_block4(x))  # 6 -> 3
        x = x.view(-1, 256 * 3 * 3)
        x = self.fc1(x)
        x = nn.LeakyReLU(0.1)(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = nn.LeakyReLU(0.1)(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

model = ER_CNN()

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100*correct/total:.2f}%')

Epoch [1/20], Loss: 1.7143, Accuracy: 30.38%
Epoch [2/20], Loss: 1.4606, Accuracy: 43.41%
Epoch [3/20], Loss: 1.3558, Accuracy: 47.72%
Epoch [4/20], Loss: 1.2824, Accuracy: 51.15%
Epoch [5/20], Loss: 1.2349, Accuracy: 53.32%
Epoch [6/20], Loss: 1.1947, Accuracy: 54.57%
Epoch [7/20], Loss: 1.1664, Accuracy: 55.67%
Epoch [8/20], Loss: 1.1414, Accuracy: 56.91%
Epoch [9/20], Loss: 1.1230, Accuracy: 57.68%
Epoch [10/20], Loss: 1.0985, Accuracy: 58.28%
Epoch [11/20], Loss: 1.0849, Accuracy: 58.77%
Epoch [12/20], Loss: 1.0717, Accuracy: 59.55%
Epoch [13/20], Loss: 1.0556, Accuracy: 59.93%
Epoch [14/20], Loss: 1.0424, Accuracy: 60.45%
Epoch [15/20], Loss: 1.0276, Accuracy: 61.05%
Epoch [16/20], Loss: 1.0098, Accuracy: 61.87%
Epoch [17/20], Loss: 0.9979, Accuracy: 62.22%
Epoch [18/20], Loss: 0.9898, Accuracy: 62.41%
Epoch [19/20], Loss: 0.9827, Accuracy: 62.89%
Epoch [20/20], Loss: 0.9616, Accuracy: 63.68%


In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%')

    model.eval()
    test_correct = 0
    test_total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

    test_accuracy = 100 * test_correct / test_total
    print(f'Test Accuracy after Epoch {epoch+1}: {test_accuracy:.2f}%')

    model.train()

Epoch [1/20], Train Loss: 1.6763, Train Accuracy: 33.10%
Test Accuracy after Epoch 1: 43.33%
Epoch [2/20], Train Loss: 1.4858, Train Accuracy: 42.47%
Test Accuracy after Epoch 2: 46.67%
Epoch [3/20], Train Loss: 1.3977, Train Accuracy: 46.54%
Test Accuracy after Epoch 3: 50.28%
Epoch [4/20], Train Loss: 1.3403, Train Accuracy: 48.90%
Test Accuracy after Epoch 4: 51.59%
Epoch [5/20], Train Loss: 1.2949, Train Accuracy: 51.02%
Test Accuracy after Epoch 5: 51.35%
Epoch [6/20], Train Loss: 1.2621, Train Accuracy: 51.96%
Test Accuracy after Epoch 6: 52.76%
Epoch [7/20], Train Loss: 1.2370, Train Accuracy: 53.13%
Test Accuracy after Epoch 7: 54.14%
Epoch [8/20], Train Loss: 1.2207, Train Accuracy: 53.54%
Test Accuracy after Epoch 8: 55.24%
Epoch [9/20], Train Loss: 1.1995, Train Accuracy: 54.25%
Test Accuracy after Epoch 9: 55.96%
Epoch [10/20], Train Loss: 1.1801, Train Accuracy: 55.21%
Test Accuracy after Epoch 10: 56.41%
Epoch [11/20], Train Loss: 1.1634, Train Accuracy: 55.79%
Test Accur

In [30]:
uploaded = files.upload()

Saving actor.jpg to actor.jpg


In [34]:
from facenet_pytorch import MTCNN

img = Image.open('/content/test1.jpg')
face = MTCNN(img)
face = transform(img)
face = face.unsqueeze(0)

In [35]:
face = face.to(device)

In [36]:
classes = {
    0: 'Angry',
    1: 'Disgust',
    2: 'Fear',
    3: 'Happy',
    4: 'Neutral',
    5: 'Sad',
    6: 'Surprise'
}

In [37]:
model.eval()
with torch.no_grad():
    output = model(face)
    _, predicted = torch.max(output, 1)
print('Predicted emotion:', classes[predicted.item()])

Predicted emotion: Happy
